In [1]:
# !pip install --upgrade transformers
# !pip install simpletransformers
# # memory footprint support libraries/code
# !ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
# !pip install gputil
# !pip install psutil
# !pip install humanize
# !pip install sentence_transformers

In [ ]:
import psutil
import humanize
import os
import GPUtil as GPU

GPUs = GPU.getGPUs()
gpu = GPUs[0]
def printm():
    process = psutil.Process(os.getpid())
    print("Gen RAM Free: " + humanize.naturalsize(psutil.virtual_memory().available), " |     Proc size: " + humanize.naturalsize(process.memory_info().rss))
    print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total     {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()


Gen RAM Free: 26.4 GB  |     Proc size: 110.9 MB
GPU RAM Free: 16130MB | Used: 0MB | Util   0% | Total     16130MB


In [ ]:
import numpy as np
import pandas as pd
from google.colab import files
from tqdm import tqdm
import warnings
warnings.simplefilter('ignore')
import gc
from scipy.special import softmax
from simpletransformers.classification import ClassificationModel
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
import sklearn
from sklearn.metrics import log_loss
from sklearn.metrics import *
from sklearn.model_selection import *
import re
import random
import torch
pd.options.display.max_colwidth = 200

def seed_all(seed_value):
    random.seed(seed_value) # Python
    np.random.seed(seed_value) # cpu vars
    torch.manual_seed(seed_value) # cpu  vars
    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value) # gpu vars
        torch.backends.cudnn.deterministic = True  #needed
        torch.backends.cudnn.benchmark = False


### Reading the train and test data and creating a new feature column

In [ ]:
train_df = pd.read_csv(r'/content/train.csv',sep=";",encoding='utf-8')
test_df  = pd.read_csv(r'/content/test_reduced.csv',sep=";",encoding='utf-8')
train_df.loc[train_df['MailSubject'].isnull(),'MailSubject']=''
train_df['Text'] = train_df['MailSubject']+' '+train_df['MailTextBody']
test_df['Text']  = test_df['MailSubject']+' '+test_df['MailTextBody']

In [ ]:
############### Basic preprocessing involved completed  ##################

train_df['Text']=train_df['Text'].str.replace('\\r',' ')
train_df['Text']=train_df['Text'].str.replace('\\n','')
train_df['Text']=train_df['Text'].str.replace('\r','')
train_df['Text']=train_df['Text'].str.replace('\n','')
train_df['Text']=train_df['Text'].str.replace('  ','')
train_df['Text']=train_df['Text'].str.lower()
train_df['Text']=train_df['Text'].str.replace('&nbsp;','')
train_df['Text']=train_df['Text'].str.strip()
train_df['Text']=train_df['Text'].astype(str)


test_df['Text']=test_df['Text'].str.replace('\\r',' ')
test_df['Text']=test_df['Text'].str.replace('\\n','')
test_df['Text']=test_df['Text'].str.replace('\r','')
test_df['Text']=test_df['Text'].str.replace('\n','')
test_df['Text']=test_df['Text'].str.replace('  ','')
test_df['Text']=test_df['Text'].str.lower()
test_df['Text']=test_df['Text'].str.replace('&nbsp;','')
test_df['Text']=test_df['Text'].str.strip()
test_df['Text']=test_df['Text'].astype(str)

### Loading a pretrained sentence embedding model trained on German corpus

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('T-Systems-onsite/cross-en-de-roberta-sentence-transformer')

Exception when trying to download https://sbert.net/models/T-Systems-onsite/cross-en-de-roberta-sentence-transformer.zip. Response 404


In [ ]:
trainenc = model.encode(train_df['Text'].values)
testenc  = model.encode(test_df['Text'].values)

### Getting data ready for feature selection 

In [ ]:
trainencdf = pd.DataFrame(trainenc,columns=['F_'+str(i+1) for i in range(trainenc.shape[1])])
testencdf = pd.DataFrame(testenc,columns=['F_'+str(i+1) for i in range(testenc.shape[1])])

In [ ]:
trainencdf.head(2)

,F_1,F_2,F_3,F_4,F_5,F_6,F_7,F_8,F_9,F_10,F_11,F_12,F_13,F_14,F_15,F_16,F_17,F_18,F_19,F_20,F_21,F_22,F_23,F_24,F_25,F_26,F_27,F_28,F_29,F_30,F_31,F_32,F_33,F_34,F_35,F_36,F_37,F_38,F_39,F_40,...,F_729,F_730,F_731,F_732,F_733,F_734,F_735,F_736,F_737,F_738,F_739,F_740,F_741,F_742,F_743,F_744,F_745,F_746,F_747,F_748,F_749,F_750,F_751,F_752,F_753,F_754,F_755,F_756,F_757,F_758,F_759,F_760,F_761,F_762,F_763,F_764,F_765,F_766,F_767,F_768
0,0.034012,0.075266,-0.037337,0.162976,0.185367,0.176372,0.037787,-0.029154,-0.118751,-0.031563,0.066883,-0.087207,-0.123629,-0.325108,0.013429,-0.160016,-0.033490,0.463228,0.540422,0.053210,0.091479,-0.086251,-0.033609,0.036056,0.387035,0.034744,0.000547,-0.259147,0.215215,0.310396,0.099381,-0.552036,0.086600,-0.167696,0.035786,0.179524,0.051584,0.076566,-0.816912,-0.080277,...,0.321992,-0.128655,-0.117164,-0.249862,0.023743,0.019265,-0.098947,-0.097178,-0.208129,0.571833,0.405610,0.085481,0.086475,0.064139,0.011750,0.107537,-0.077895,0.139906,-0.098872,0.282352,-0.661906,-0.590695,0.234361,0.280968,-0.238311,0.309138,-0.089454,0.162467,-0.111809,0.091559,-0.229673,-0.095214,0.166922,0.446252,-0.172275,-0.137226,0.003556,-0.231111,-0.087406,0.155381
1,-0.042011,-0.116525,-0.013540,-0.632233,0.066639,-0.239721,0.258594,0.021547,-0.082815,-0.091217,0.675750,-0.165820,-0.098464,0.307436,0.382840,0.294781,0.166198,0.092967,0.441233,-0.078812,-0.028009,0.353567,-0.217048,-0.219686,0.550215,0.047500,0.231025,-0.176146,-0.322216,-0.312013,0.044913,0.292335,-0.005531,0.347323,-0.059626,-0.149310,-0.261692,-0.468779,0.018533,-0.106825,...,0.261619,0.161311,0.028805,0.015440,0.433908,0.240926,0.350505,-0.042932,-0.181641,-0.068372,0.103913,-0.088435,-0.442536,0.126901,0.125773,0.300356,0.109496,-0.254320,-0.092179,0.470452,-0.409802,-0.264961,-0.298801,0.244276,-0.168392,-0.287207,0.001203,0.292325,-0.128439,0.004491,-0.141996,0.041622,-0.189335,0.052094,-0.051398,0.001455,0.003675,-0.018652,-0.096094,0.280907


In [ ]:
trainencdf['ServiceProcessed']=train_df['ServiceProcessed']
trainencdf.shape

(2845, 769)

### We will use a boosting algorithm that will be used for one vs all binaryclassification for each feature and determine the feature importances needed for each class

In [ ]:
import lightgbm as lgb

feature_stack=pd.DataFrame()

for i in trainencdf['ServiceProcessed'].unique():
  trainencdf['Target']=np.where(trainencdf['ServiceProcessed']==i,1,0)
  cols=[col for col in trainencdf.columns if col not in ['ServiceProcessed','Target']]
  X=trainencdf[cols]
  y=trainencdf['Target']
  model=lgb.LGBMClassifier(random_state=202020)
  model.fit(X,y)
  fea_imp = pd.DataFrame({'imp': model.feature_importances_, 'col': cols})
  fea_imp.sort_values(by='imp',ascending=False,inplace=True)
  fea_imp=fea_imp.head(100)
  feature_stack[str(i)]=fea_imp.col.values
  print(i,feature_stack.shape)

EDA_S_BA_2FA (100, 1)
EDA_S_Order Management (100, 2)
EDA_S_Peripheriegeräte (100, 3)
EDA_S_Benutzerunterstützung (100, 4)
EDA_ANW_SysP eDoc (100, 5)
EDA_S_Betrieb Übermitttlungssysteme (100, 6)
EDA_S_IT Sicherheit (100, 7)
EDA_S_Raumbewirtschaftung (100, 8)
EDA_S_BA_Mailbox (100, 9)
EDA_S_BA_UCC_IVR (100, 10)
EDA_S_APS_Peripherie (100, 11)
EDA_S_BA_ServerAusland (100, 12)
EDA_ANW_Intranet/Collaboration EDA (100, 13)
EDA_S_BA_UCC_Benutzertelefonie (100, 14)
EDA_S_APS_PC (100, 15)
EDA_S_Mobile Kommunikation (100, 16)
EDA_S_BA_Account (100, 17)
EDA_S_Netzdrucker (100, 18)
EDA_S_APS_OS_BasisSW (100, 19)
EDA_ANW_DMS Fabasoft eGov Suite (100, 20)
EDA_ANW_Office Manager (100, 21)
EDA_S_APS_Monitor (100, 22)
EDA_S_Zusätzliche Software (100, 23)
EDA_ANW_Zentrale Raumreservation EDA (100, 24)
EDA_S_Netzwerk Ausland (100, 25)
EDA_ANW_CodX PostOffice (100, 26)
_Pending (100, 27)
EDA_ANW_SAP Services (100, 28)
EDA_S_APS (100, 29)
EDA_S_BA_Datenablage (100, 30)
EDA_S_Büroautomation (100, 31)
EDA_AN

In [ ]:
feature_stack

,EDA_S_BA_2FA,EDA_S_Order Management,EDA_S_Peripheriegeräte,EDA_S_Benutzerunterstützung,EDA_ANW_SysP eDoc,EDA_S_Betrieb Übermitttlungssysteme,EDA_S_IT Sicherheit,EDA_S_Raumbewirtschaftung,EDA_S_BA_Mailbox,EDA_S_BA_UCC_IVR,EDA_S_APS_Peripherie,EDA_S_BA_ServerAusland,EDA_ANW_Intranet/Collaboration EDA,EDA_S_BA_UCC_Benutzertelefonie,EDA_S_APS_PC,EDA_S_Mobile Kommunikation,EDA_S_BA_Account,EDA_S_Netzdrucker,EDA_S_APS_OS_BasisSW,EDA_ANW_DMS Fabasoft eGov Suite,EDA_ANW_Office Manager,EDA_S_APS_Monitor,EDA_S_Zusätzliche Software,EDA_ANW_Zentrale Raumreservation EDA,EDA_S_Netzwerk Ausland,EDA_ANW_CodX PostOffice,_Pending,EDA_ANW_SAP Services,EDA_S_APS,EDA_S_BA_Datenablage,EDA_S_Büroautomation,EDA_ANW_ARS Remedy,EDA_S_BA_RemoteAccess,EDA_ANW_Internet EDA,EDA_ANW_at Honorarvertretung,EDA_ANW_IAM Tool EDA,EDA_ANW_EDAContacts,EDA_S_Backup & Restore,EDA_ANW_ZACWEB,EDA_ANW_CH@World (MOSS),EDA_ANW_eVERA,EDA_S_BA_Internetzugriff,EDA_ANW_ORBIS,EDA_S_Netzwerk Inland,EDA_ANW_EDA PWC Tool,EDA_ANW_EDAssist+,EDA_ANW_Zeiterfassung SAP,EDA_ANW_ITDoc Sharepoint,EDA_ANW_MOVE!,EDA_S_Arbeitsplatzdrucker,EDA_ANW_Plato-HH,EDA_ANW_NOS:4,EDA_ANW_Reisehinweise,EDA_ANW_ARIS (EDA Scout),EDA_ANW_FDFA Security App
0,F_144,F_138,F_464,F_179,F_59,F_624,F_204,F_331,F_768,F_404,F_609,F_481,F_209,F_358,F_673,F_193,F_134,F_232,F_80,F_36,F_93,F_379,F_343,F_1,F_186,F_359,F_728,F_250,F_309,F_113,F_374,F_648,F_478,F_483,F_243,F_196,F_245,F_1,F_542,F_765,F_1,F_390,F_1,F_18,F_1,F_1,F_1,F_1,F_613,F_1,F_726,F_1,F_1,F_1,F_1
1,F_612,F_249,F_43,F_255,F_544,F_396,F_736,F_758,F_307,F_433,F_309,F_265,F_271,F_361,F_472,F_549,F_450,F_610,F_713,F_355,F_570,F_591,F_645,F_214,F_102,F_1,F_139,F_725,F_128,F_159,F_721,F_423,F_106,F_359,F_287,F_410,F_363,F_95,F_293,F_3,F_128,F_271,F_2,F_164,F_21,F_2,F_2,F_288,F_209,F_2,F_234,F_4,F_2,F_189,F_625
2,F_59,F_649,F_721,F_52,F_625,F_138,F_287,F_698,F_14,F_642,F_216,F_366,F_575,F_125,F_242,F_212,F_626,F_345,F_156,F_472,F_358,F_487,F_138,F_303,F_383,F_170,F_676,F_677,F_425,F_530,F_522,F_618,F_110,F_235,F_393,F_671,F_468,F_626,F_116,F_711,F_466,F_375,F_26,F_1,F_414,F_3,F_3,F_764,F_70,F_3,F_603,F_2,F_3,F_343,F_295
3,F_84,F_557,F_438,F_227,F_153,F_212,F_235,F_588,F_736,F_763,F_31,F_360,F_236,F_1,F_168,F_155,F_669,F_142,F_189,F_618,F_144,F_132,F_151,F_289,F_633,F_406,F_420,F_358,F_111,F_739,F_308,F_675,F_27,F_654,F_548,F_696,F_136,F_306,F_96,F_189,F_2,F_600,F_565,F_697,F_516,F_179,F_143,F_596,F_567,F_6,F_697,F_3,F_435,F_2,F_753
4,F_338,F_315,F_476,F_595,F_225,F_170,F_566,F_369,F_752,F_126,F_11,F_116,F_616,F_355,F_483,F_141,F_246,F_149,F_536,F_210,F_567,F_679,F_333,F_47,F_329,F_2,F_21,F_708,F_5,F_4,F_755,F_577,F_31,F_342,F_626,F_409,F_175,F_358,F_520,F_393,F_386,F_579,F_491,F_185,F_2,F_298,F_38,F_16,F_402,F_4,F_75,F_169,F_4,F_318,F_97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,F_354,F_533,F_515,F_1,F_158,F_151,F_721,F_730,F_294,F_15,F_41,F_152,F_239,F_171,F_528,F_62,F_4,F_50,F_548,F_610,F_464,F_543,F_633,F_323,F_482,F_257,F_198,F_101,F_653,F_722,F_719,F_18,F_23,F_115,F_78,F_699,F_39,F_667,F_317,F_240,F_441,F_711,F_292,F_116,F_594,F_642,F_493,F_693,F_404,F_576,F_10,F_78,F_557,F_244,F_717
96,F_419,F_479,F_359,F_510,F_588,F_309,F_160,F_427,F_683,F_77,F_581,F_650,F_373,F_527,F_603,F_123,F_200,F_567,F_312,F_197,F_720,F_633,F_552,F_322,F_573,F_246,F_133,F_198,F_221,F_119,F_181,F_273,F_680,F_253,F_351,F_258,F_705,F_620,F_406,F_144,F_428,F_673,F_440,F_158,F_156,F_385,F_540,F_46,F_25,F_577,F_224,F_68,F_537,F_150,F_39
97,F_3,F_196,F_375,F_15,F_357,F_276,F_724,F_502,F_336,F_449,F_758,F_127,F_470,F_277,F_94,F_617,F_656,F_292,F_483,F_481,F_466,F_220,F_355,F_700,F_1,F_227,F_164,F_541,F_285,F_229,F_338,F_432,F_340,F_362,F_345,F_456,F_281,F_179,F_348,F_359,F_417,F_581,F_545,F_723,F_491,F_80,F_541,F_147,F_640,F_578,F_354,F_70,F_538,F_388,F_626
98,F_170,F_206,F_542,F_56,F_398,F_701,F_718,F_11

In [ ]:
def capture_similar_feats(a,b):
  '''
    This function will capture percentage of similar features for two vectors 
  '''
  nr = len(list(set(a) & set(b)))
  dr = 100 
  return nr/dr

In [ ]:
######### We will create a similarity matrix based on the feature similarity of two classes. Since the matrix will be a symmetric matrix, we will prefer to calculate the upper diagonal

dim = 55
similarity_matrix = np.zeros((55,55))
for i in range(similarity_matrix.shape[0]):
  col = feature_stack.columns[i]

  for j in range(i+1,similarity_matrix.shape[1]):
    col2comp = feature_stack.columns[j]
    similarity_matrix[i,j]=capture_similar_feats(feature_stack[col],feature_stack[col2comp])

In [ ]:
clusterdf=pd.DataFrame(similarity_matrix,columns=feature_stack.columns,index= feature_stack.columns)

In [ ]:
clusterdf.head()

,EDA_S_BA_2FA,EDA_S_Order Management,EDA_S_Peripheriegeräte,EDA_S_Benutzerunterstützung,EDA_ANW_SysP eDoc,EDA_S_Betrieb Übermitttlungssysteme,EDA_S_IT Sicherheit,EDA_S_Raumbewirtschaftung,EDA_S_BA_Mailbox,EDA_S_BA_UCC_IVR,EDA_S_APS_Peripherie,EDA_S_BA_ServerAusland,EDA_ANW_Intranet/Collaboration EDA,EDA_S_BA_UCC_Benutzertelefonie,EDA_S_APS_PC,EDA_S_Mobile Kommunikation,EDA_S_BA_Account,EDA_S_Netzdrucker,EDA_S_APS_OS_BasisSW,EDA_ANW_DMS Fabasoft eGov Suite,EDA_ANW_Office Manager,EDA_S_APS_Monitor,EDA_S_Zusätzliche Software,EDA_ANW_Zentrale Raumreservation EDA,EDA_S_Netzwerk Ausland,EDA_ANW_CodX PostOffice,_Pending,EDA_ANW_SAP Services,EDA_S_APS,EDA_S_BA_Datenablage,EDA_S_Büroautomation,EDA_ANW_ARS Remedy,EDA_S_BA_RemoteAccess,EDA_ANW_Internet EDA,EDA_ANW_at Honorarvertretung,EDA_ANW_IAM Tool EDA,EDA_ANW_EDAContacts,EDA_S_Backup & Restore,EDA_ANW_ZACWEB,EDA_ANW_CH@World (MOSS),EDA_ANW_eVERA,EDA_S_BA_Internetzugriff,EDA_ANW_ORBIS,EDA_S_Netzwerk Inland,EDA_ANW_EDA PWC Tool,EDA_ANW_EDAssist+,EDA_ANW_Zeiterfassung SAP,EDA_ANW_ITDoc Sharepoint,EDA_ANW_MOVE!,EDA_S_Arbeitsplatzdrucker,EDA_ANW_Plato-HH,EDA_ANW_NOS:4,EDA_ANW_Reisehinweise,EDA_ANW_ARIS (EDA Scout),EDA_ANW_FDFA Security App
EDA_S_BA_2FA,0.0,0.12,0.07,0.12,0.15,0.16,0.13,0.18,0.14,0.13,0.16,0.11,0.14,0.20,0.17,0.16,0.11,0.16,0.13,0.13,0.12,0.15,0.20,0.14,0.14,0.15,0.16,0.13,0.10,0.16,0.18,0.13,0.14,0.16,0.11,0.17,0.18,0.19,0.16,0.21,0.12,0.15,0.16,0.14,0.17,0.15,0.17,0.15,0.12,0.18,0.07,0.16,0.19,0.15,0.16
EDA_S_Order Management,0.0,0.00,0.09,0.12,0.14,0.09,0.14,0.08,0.09,0.19,0.15,0.10,0.08,0.12,0.12,0.16,0.15,0.14,0.13,0.17,0.16,0.17,0.09,0.14,0.10,0.13,0.19,0.15,0.13,0.11,0.12,0.16,0.13,0.19,0.13,0.16,0.16,0.15,0.15,0.13,0.15,0.05,0.13,0.11,0.12,0.13,0.22,0.11,0.11,0.23,0.09,0.09,0.21,0.15,0.15
EDA_S_Peripheriegeräte,0.0,0.00,0.00,0.08,0.19,0.13,0.20,0.09,0.19,0.13,0.15,0.16,0.18,0.15,0.15,0.19,0.10,0.14,0.10,0.11,0.18,0.16,0.11,0.15,0.19,0.18,0.15,0.22,0.13,0.14,0.15,0.14,0.10,0.22,0.11,0.12,0.11,0.13,0.16,0.11,0.10,0.14,0.16,0.19,0.15,0.14,0.14,0.18,0.11,0.14,0.16,0.19,0.18,0.16,0.17
EDA_S_Benutzerunterstützung,0.0,0.00,0.00,0.00,0.12,0.12,0.22,0.10,0.14,0.21,0.16,0.15,0.13,0.12,0.20,0.13,0.13,0.14,0.15,0.16,0.14,0.08,0.15,0.13,0.14,0.19,0.08,0.14,0.13,0.12,0.14,0.13,0.12,0.19,0.10,0.12,0.10,0.13,0.18,0.14,0.15,0.15,0.12,0.17,0.09,0.18,0.10,0.12,0.12,0.11,0.14,0.17,0.08,0.19,0.13
EDA_ANW_SysP eDoc,0.0,0.00,0.00,0.00,0.00,0.13,0.10,0.17,0.14,0.15,0.13,0.10,0.15,0.11,0.14,0.14,0.17,0.13,0.17,0.11,0.19,0.12,0.12,0.05,0.15,0.13,0.14,0.12,0.11,0.18,0.16,0.12,0.12,0.14,0.10,0.12,0.18,0.10,0.14,0.12,0.15,0.16,0.08,0.12,0.10,0.13,0.10,0.19,0.09,0.10,0.11,0.06,0.08,0.09,0.13


In [ ]:
########## Pairing up similar features with one other. We will now combine different features based on their similarity scores with each other.

clusterdf1=clusterdf.copy()
merged_services={}
while len(clusterdf1.columns)>0:
  for i in clusterdf1.columns:
    a=clusterdf1[i].nlargest(4).index.tolist()
    merged_services[i]=clusterdf1[i].nlargest(4).index.tolist()
    clusterdf1=clusterdf1.drop(a,axis=1).drop(a,axis=0)
    print(clusterdf1.shape)
    break


(51, 51)
(47, 47)
(43, 43)
(39, 39)
(35, 35)
(31, 31)
(27, 27)
(23, 23)
(19, 19)
(15, 15)
(11, 11)
(7, 7)
(3, 3)
(0, 0)


In [ ]:
###### Final merged dataset
merged_services_df=pd.DataFrame.from_dict(merged_services, orient='index')
merged_services_df.head()

,0,1,2,3
EDA_S_BA_2FA,EDA_S_BA_2FA,EDA_S_Order Management,EDA_S_Peripheriegeräte,EDA_S_Benutzerunterstützung
EDA_ANW_SysP eDoc,EDA_ANW_SysP eDoc,EDA_S_Betrieb Übermitttlungssysteme,EDA_S_IT Sicherheit,EDA_S_Raumbewirtschaftung
EDA_S_BA_Mailbox,EDA_S_BA_Mailbox,EDA_S_BA_UCC_IVR,EDA_S_APS_Peripherie,EDA_S_BA_ServerAusland
EDA_ANW_Intranet/Collaboration EDA,EDA_ANW_Intranet/Collaboration EDA,EDA_S_BA_UCC_Benutzertelefonie,EDA_S_APS_PC,EDA_S_Mobile Kommunikation
EDA_S_BA_Account,EDA_S_BA_Account,EDA_S_Netzdrucker,EDA_S_APS_OS_BasisSW,EDA_ANW_DMS Fabasoft eGov Suite
